In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from unidecode import unidecode

import pandas as pd
import numpy as np

import os
import re
import pickle
import json
import copy
import datetime
import unidecode
import bs4

In [2]:
def date_decoder(date):
    if date == '11-05-19.html': return '2019-05-11'
    if '-' in date:
        if 'html' in date: date = date[:-5]
        if  re.match('\d{2}-\d{2}-\d{2}$',date):
            return re.sub('19','2019','-'.join(date.split('-')[::-1]))

        return '-'.join(date.split('-')[::-1])
    else:
        months_short = {'jan':'01','feb':'02','mar':'03','apr':'04','may':'05','jun':'06','jul':'07','aug':'08','sep':'09'}
        month = re.search('(\D+)',date)[0]
        date = re.sub(month,f'-{months_short[month]}-',date)
        if 'html' in date: date = date[:-5]
        return '-'.join(date.split('-')[::-1])

In [3]:
file_names = [[x,date_decoder(x)] for x in os.listdir("E:\\Downloads\\ufc_site_shots") if 'html' in x]

In [4]:
sorted(file_names,key=lambda x:x[1])

[['01-08-2018.html', '2018-08-01'],
 ['06-08-2018.html', '2018-08-06'],
 ['29-08-2018.html', '2018-08-29'],
 ['12-09-2018.html', '2018-09-12'],
 ['19-09-2018.html', '2018-09-19'],
 ['29-09-2018.html', '2018-09-29'],
 ['08-10-2018.html', '2018-10-08'],
 ['31-10-2018.html', '2018-10-31'],
 ['07-11-2018.html', '2018-11-07'],
 ['13-11-2018.html', '2018-11-13'],
 ['21-11-2018.html', '2018-11-21'],
 ['12-12-2018.html', '2018-12-12'],
 ['19-12-2018.html', '2018-12-19'],
 ['02jan2019.html', '2019-01-02'],
 ['09jan2019.html', '2019-01-09'],
 ['16jan2019.html', '2019-01-16'],
 ['09feb2019.html', '2019-02-09'],
 ['27mar2019.html', '2019-03-27'],
 ['29mar2019.html', '2019-03-29'],
 ['06apr2019.html', '2019-04-06'],
 ['17apr2019.html', '2019-04-17'],
 ['21apr2019.html', '2019-04-21'],
 ['30apr2019.html', '2019-04-30'],
 ['11-05-19.html', '2019-05-11'],
 ['13-05-19.html', '2019-05-13'],
 ['19-05-2019.html', '2019-05-19'],
 ['03jun2019.html', '2019-06-03']]

In [5]:
def clean(string):
    return re.sub('\s+',' ',string)

In [6]:
def old_site(link_and_name:list):
    fighters = []
    site = BeautifulSoup(open(f"E:\\Downloads\\ufc_site_shots\\{link_and_name[0]}",'rb'),'html.parser')
    
    for fighters_list in site.find_all('div',class_='ranking-list tall'):
        if fighters_list.find('div','home-error-message'): continue
        wclass = fighters_list.find('div',class_='weight-class-name').text.strip()
        if wclass != 'Pound-for-Pound':
            if fighters_list.find('div',id='champion-fighter'):
                champion = fighters_list.find('div',id='champion-fighter').a.text.strip()
                fighters.append([link_and_name[1],wclass,champion,0])
            
        for fighter_in_list in fighters_list.find_all('tr'):
            fighters.append([link_and_name[1],
                                 wclass,
                                 fighter_in_list.find('td',class_="name-column").find('a').text.strip(),
                                 fighter_in_list.find('td',class_="number-column").text.strip()])
    return fighters

In [7]:
def new_site(link_and_name:list):
    fighters = []
    site = BeautifulSoup(open(f"E:\\Downloads\\ufc_site_shots\\{link_and_name[0]}",'rb'),'html.parser')
    
    for fighters_list in site.find('div','view-athlete-rankings').find_all('div','view-grouping'):
        if fighters_list.find('h4').find('span'):
            wclass = 'Pound-for-Pound'
            fighters.append([link_and_name[1],wclass,fighters_list.find('h5').text.strip(),1])
            
        else:
            wclass = fighters_list.find('h4').text
            if fighters_list.find('h5'):
                fighters.append([link_and_name[1],wclass,fighters_list.find('h5').a.text,0])
        
        for fighter in fighters_list.find_all('tr'):
            rank = fighter.find('td').text.strip()
            name = unidecode.unidecode(fighter.a.text)
            fighters.append([link_and_name[1],wclass,name,rank])
    
    return fighters

In [8]:
all_fighters_rankings = []
for site_name in sorted(file_names,key=lambda x:x[1]):
    print(site_name)
    #new site
    if datetime.date(*(int(x) for x in site_name[1].split('-'))) > datetime.date(2018,11,10):
        all_fighters_rankings.append(new_site(site_name))
    else:
        all_fighters_rankings.append(old_site(site_name))

['01-08-2018.html', '2018-08-01']
['06-08-2018.html', '2018-08-06']
['29-08-2018.html', '2018-08-29']
['12-09-2018.html', '2018-09-12']
['19-09-2018.html', '2018-09-19']
['29-09-2018.html', '2018-09-29']
['08-10-2018.html', '2018-10-08']
['31-10-2018.html', '2018-10-31']
['07-11-2018.html', '2018-11-07']
['13-11-2018.html', '2018-11-13']
['21-11-2018.html', '2018-11-21']
['12-12-2018.html', '2018-12-12']
['19-12-2018.html', '2018-12-19']
['02jan2019.html', '2019-01-02']
['09jan2019.html', '2019-01-09']
['16jan2019.html', '2019-01-16']
['09feb2019.html', '2019-02-09']
['27mar2019.html', '2019-03-27']
['29mar2019.html', '2019-03-29']
['06apr2019.html', '2019-04-06']
['17apr2019.html', '2019-04-17']
['21apr2019.html', '2019-04-21']
['30apr2019.html', '2019-04-30']
['11-05-19.html', '2019-05-11']
['13-05-19.html', '2019-05-13']
['19-05-2019.html', '2019-05-19']
['03jun2019.html', '2019-06-03']


In [9]:
concated = []
for fighter_list in all_fighters_rankings:
    concated += fighter_list

In [10]:
new_ranks = pd.DataFrame(concated,columns=['date','weightclass','fighter','rank'])

In [11]:
old_ranks = pd.read_csv('rankings_history.csv')

In [12]:
all_ranks = pd.concat([old_ranks,new_ranks])

In [13]:
all_ranks = all_ranks.reset_index(drop=True)

In [14]:
all_ranks.to_csv('full_ranking.csv')

In [15]:
all_ranks = pd.read_csv('full_ranking.csv', index_col='Unnamed: 0')

In [16]:
all_ranks.tail()

,date,weightclass,fighter,rank
31046,2019-06-03,Women's Bantamweight,Tonya Evinger,12
31047,2019-06-03,Women's Bantamweight,Lina Lansberg,13
31048,2019-06-03,Women's Bantamweight,Bethe Correia,14
31049,2019-06-03,Women's Bantamweight,Sijara Eubanks,15
31050,2019-06-03,Women's Featherweight,Amanda Nunes,0


In [17]:
def Name_correction(string):
    if re.match('James Te-Huna',string): return 'James Te Huna'
    if re.match('TJ Dillashaw',string): return 'T.J. Dillashaw'
    if re.match('Costa Philippou',string): return 'Costas Philippou'
    if re.match('Jessica-Rose Clark',string): return 'Jessica Rose Clark'
    if re.match('Junior dos Santos',string): return 'Junior Dos Santos'
    if re.match('Rafael dos Anjos',string): return 'Rafael Dos Anjos'
    if re.match('Rafael Assuncao',string): return 'Raphael Assuncao'
    if re.match('Elizeu Zaleski dos Santos',string): return 'Elizeu dos Santos'
    if re.match('Alexey Oliynyk',string): return 'Aleksei Oleinik'
    if re.match('Colby',string): return 'Colby Covington'
    if re.match('Timothy Elliott',string): return 'Tim Elliott'
    if re.match('Rogerio Nogueira',string): return 'Antonio Rogerio Nogueira'
    if re.match('Minotauro Nogueira',string): return 'Antonio Rodrigo Nogueira'
    return string

In [18]:
all_ranks['fighter'] = all_ranks['fighter'].apply(lambda x: Name_correction(x))

In [19]:
ranks_pivoted = all_ranks.groupby(['fighter','weightclass','date']).agg({'rank':'max'}).unstack()

In [20]:
ranks_pivoted

rank                        \
date                                        2013-02-04 2013-02-11 2013-02-12   
fighter               weightclass                                              
Aisling Daly          Women's Strawweight          NaN        NaN        NaN   
Al Iaquinta           Lightweight                  NaN        NaN        NaN   
Alan Belcher          Middleweight                 9.0        9.0        9.0   
Albert Tumenov        Welterweight                 NaN        NaN        NaN   
Alejandro Perez       Bantamweight                 NaN        NaN        NaN   
Aleksei Oleinik       Heavyweight                  NaN        NaN        NaN   
Alex Caceres          Bantamweight                 NaN        NaN        NaN   
Alex Oliveira         Welterweight                 NaN        NaN        NaN   
Alex Perez            Flyweight                    NaN        NaN        NaN   
Alexa Grasso          Women's Strawweight          NaN        NaN        NaN   
Alexander Gustafsson  Light Heavyweight            3.0        3.0        3.0   
                      Pound-for-Pound              NaN        NaN        NaN   
Alexander Hernandez   Lightweight                  NaN        NaN        NaN   
Alexander Volkanovski Featherweight                NaN        NaN        NaN   
Alexander Volkov      Heavyweight                  NaN        NaN        NaN   
Alexandre Pantoja     Flyweight                    NaN        NaN        NaN   
Alexis Davis          Women's Bantamweight         NaN        NaN        NaN   
                      Women's Flyweight            NaN        NaN        NaN   
Ali Bagautinov        Flyweight                    NaN        NaN        NaN   
Alistair Overeem      Heavyweight                  6.0        6.0        6.0   
Aljamain Sterling     Bantamweight                 NaN        NaN        NaN   
Amanda Nunes          Pound-for-Pound              NaN        NaN        NaN   
                      Women's Bantamweight         NaN        NaN        NaN   
                      Women's Featherweight        NaN        NaN        NaN   
Anderson Silva        Middleweight                 0.0        0.0        0.0   
                      Pound-for-Pound              1.0        1.0        1.0   
Andre Fili            Featherweight                NaN        NaN        NaN   
Andrea Lee            Women's Flyweight            NaN        NaN        NaN   
Andrei Arlovski       Heavyweight                  NaN        NaN        NaN   
Angela Hill           Women's Strawweight          NaN        NaN        NaN   
...                                                ...        ...        ...   
Tyron Woodley         Welterweight                 NaN        NaN        NaN   
Tyson Pedro           Light Heavyweight            NaN        NaN        NaN   
Ulka Sasaki           Flyweight                    NaN        NaN        NaN   
Ulysses Gomez         Flyweight                   10.0       10.0       10.0   
Uriah Hall            Middleweight                 NaN        NaN        NaN   
Urijah Faber          Bantamweight                 3.0        3.0        3.0   
                      Pound-for-Pound              NaN        NaN        NaN   
Valentina Shevchenko  Pound-for-Pound              NaN        NaN        NaN   
                      Women's Bantamweight         NaN        NaN        NaN   
                      Women's Flyweight            NaN        NaN        NaN   
Valerie Letourneau    Women's Strawweight          NaN        NaN        NaN   
Vicente Luque         Welterweight                 NaN        NaN        NaN   
Vitor Belfort         Light Heavyweight            NaN        NaN        NaN   
                      Middleweight                 2.0        2.0        2.0   
                      Pound-for-Pound              NaN        NaN        NaN   
Volkan Oezdemir       Light Heavyweight            NaN        NaN        NaN   
Walt Harris           Heavyweight                 

In [21]:
def get_all_fighter_rank(fighter_names):
    all_fighters_info = dict()
    for fighter in fighter_names:
        print(fighter)
        fighters_dict = ranks_pivoted.loc[fighter].to_dict()
        all_fighters_info[fighter] = {}
        #dates
        for date in fighters_dict.keys():
            #weightclasses
            for wclass in fighters_dict[date]:
                if not np.isnan(fighters_dict[date][wclass]):
                    if date[1] not in all_fighters_info[fighter]:
                        all_fighters_info[fighter][date[1]] = {}
                    all_fighters_info[fighter][date[1]][wclass] = fighters_dict[date][wclass] 

                all_fighters_info[fighter]
    return all_fighters_info

In [22]:
info_f = get_all_fighter_rank(all_ranks['fighter'].unique())

Anderson Silva
Jon Jones
Georges St-Pierre
Jose Aldo
Benson Henderson
Cain Velasquez
Dominick Cruz
Demetrious Johnson
Frankie Edgar
Dan Henderson
Joseph Benavidez
John Dodson
Ian McCall
John Moraga
Jussier Da Silva
Louis Gaudinot
Chris Cariaso
John Lineker
Darren Uyenoyama
Ulysses Gomez
Renan Barao
Michael McDonald
Urijah Faber
Eddie Wineland
Brad Pickett
Brian Bowles
Raphael Assuncao
Scott Jorgensen
Mike Easton
Ivan Menjivar
Chad Mendes
Ricardo Lamas
Chan Sung Jung
Dennis Siver
Cub Swanson
Dustin Poirier
Nik Lentz
Erik Koch
Clay Guida
Gilbert Melendez
Anthony Pettis
Gray Maynard
Nate Diaz
Jim Miller
Donald Cerrone
TJ Grant
Rafael Dos Anjos
Joe Lauzon
Khabib Nurmagomedov
Johny Hendricks
Carlos Condit
Nick Diaz
Rory MacDonald
Demian Maia
Jake Ellenberger
Martin Kampmann
Josh Koscheck
Jon Fitch
Tarec Saffiedine
Chris Weidman
Vitor Belfort
Michael Bisping
Yushin Okami
Mark Munoz
Constantinos Philippou
Luke Rockhold
Hector Lombard
Alan Belcher
Tim Boetsch
Lyoto Machida
Alexander Gustafsson

In [23]:
json.dump(info_f,open('ranking_dict','w'))